In [1]:
import pandas as pd

import numpy as np

import warnings

import sklearn.preprocessing
import sklearn.linear_model

from numba.core.errors import NumbaDeprecationWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=NumbaDeprecationWarning)

In [2]:
df = pd.read_csv('../data/california_housing/california_housing.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df2 = pd.read_csv('../data/california_housing/california_housing.csv')
#nombre de maisons à prendre en compte (20k c'est beaucoup trop)
# quelques condition pour nettoyer la base de données
df = df.loc[df['Population']<10000]
df = df.loc[df['AveOccup']<6]
df = df.loc[df['AveBedrms']<1.5]
df = df.loc[df['HouseAge']<50]

# Seulement San Francisco :
# Latitude > 37,2
# Latitude < 38,07
# Longitude > -122,5
# Longitude < -121,75

df = df.loc[(df['Latitude']<38.07)&(df['Latitude']>37.2)]
df = df.loc[(df['Longitude']>-122.5)&(df['Longitude']<-121.75)]

save_df = df.copy()

N=200
n = N/len(df)
X_all = df.iloc[:,0:8].copy()
df1 = df.sample(frac=n, random_state=14)
df1 = df1.reset_index(drop=True)
save_df1 = df1.copy()

X = df1.iloc[:,0:8]
Y = df1.iloc[:,9]

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state = 9)
model.fit(X, Y)

SHAP = df1.iloc[:,[10,11,12,13,14,15,16,17]]

**Démarrage d'AntakIA**

In [4]:
import antakia

dataset = antakia.Dataset(X, model, y=Y, explain = SHAP)

a = antakia.AntakIA(dataset)
a.startGUI()

ModuleNotFoundError: No module named 'antakia.utils'

In [ ]:
print(a.gui.selection)

Potato:
 ------------------
       State: None 
       Number of points: 0 
       Percentage of the dataset: 0.0% 

